In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
def number_of_years(y):        #calculates the number of years of the dataset
  p=y.index[0]                 #date of first row in the dataset (datetime format)
  q=y.index[len(y)-1]          #date of last row in the dataset  (datetime format)
  return ((q-p).days+1)/365           #the difference give the number of total days (not trading days) over the total number of years in the dataset

In [6]:
# returnso=returnsh.copy()                  #this cell considers only NEGATIVE returns so as to calculate sortino ratio
# for cols in returnso.columns.tolist():
#     for i in range(0,len(df)):
#       if returnso[cols][i] > 0:
#         returnso[cols][i]=0               #Here, returnso would mean return considered for sortino ratio

In [8]:
num_portfolios =5 #50000                   #initializing number of portfolios to 50000; referred from Wang et al (2020) (science direct)
num_assets = len(df.columns)              #initializing number of stocks/assets considered in the portfolio
risk_free_rate = 0.0358                   #initializing risk free rate that will be used in calculating both the ratios (absolute value)
#referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
#In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

In [9]:
#2021_chen etal_Mean–variance portfolio optimization using machine learning-based stock price prediction
#Repeat the process 50,000times. From a statistical point of view, 50,000 random portfolios cover most possible portfolios with different weights and aresufficiently representative

In [10]:
portfolio_returns = []                    #initializing an empty list for portfolio returns
portfolio_volatility =[]                  #initializing an empty list for portfolio risk
stock_weights =[]                         #initializing an empty list for portfolio weights
#semi_deviation =[]                        #initializing an empty list for portfolio semi-deviation
sharpe =[]                                #initializing an empty list for portfolio sharpe ratio
#sortino =[]                               #initializing an empty list for portfolio sortino ratio

In [11]:
def ratio(a,b,c):                         #function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
  return (a-c)/b                          #a => annual return, c => risk_free_rate, b => deviation (standard for sharpe, semi for sortino)

In [16]:
def MVO_Sharpe(T_start_date,T_end_date,deciding_date,mvo_sharpe,lb_start_date,lb_end_date)
    df=pd.read_csv('n50.csv',parse_dates=['Date'],index_col='Date')
    df = df.loc[T_start_date : T_end_date]   #Since 2016-01-01, 5y(1238rows till 2020-12-31), + year 2021's rows
    tdf=df.copy()                  #deep copy
    df.reset_index(drop=True, inplace=True)
    trading_days=len(df)/number_of_years(tdf) 
    returnsh=df.pct_change()                  #Here, returnsh would mean return considered for sharpe ratio
    returnsh.fillna(0,inplace=True)
    covmatsh=returnsh.cov()*trading_days

    for single_portfolio in range(num_portfolios):                  #iterating forloop for 50000 times to generate 50000 portfolios
      weights = np.random.random(num_assets)                        #initializing random weights
      weights /= np.sum(weights)                                    #No Short Selling Allowed => weights add up to 1   "x = x+y" => "x+=y"    weights = weights/np.sum(weights)  
      returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
      varsh=np.dot(weights.T,np.dot(covmatsh,weights))              #calculating portfolio varience wrt calculating sharpe ratio
      #varso=np.dot(weights.T,np.dot(covmatso,weights))              #calculating portfolio varience wrt calculating sortino ratio
      volatility_temp = np.sqrt(varsh)                              #portfolio risk
      #semi_temp = np.sqrt(varso)                                    #portfolio semi-deviation
      shtemp = ratio(returns_temp,volatility_temp,risk_free_rate)   #calculating sharpe ratio
      #sotemp = ratio(returns_temp,semi_temp,risk_free_rate)         #calculating sortino ratio
      portfolio_returns.append(returns_temp)                       
      portfolio_volatility.append(volatility_temp)
      stock_weights.append(weights)
      sharpe.append(shtemp)
      #sortino.append(sotemp)
      #semi_deviation.append(semi_temp)
    portfolio = {'Returns' : portfolio_returns, 'Standard Deviation' : portfolio_volatility, 'Sharpe Ratio' : sharpe}
    for counter,symbol in enumerate(df.columns):
      portfolio[symbol + " Weight"] = [Weight[counter] for Weight in stock_weights] 
    pc = pd.DataFrame(portfolio)
    pc=pc*100                                       #Converting everything to percentage
    pc['Sharpe Ratio']=pc['Sharpe Ratio']/100       #leaving ratios as it is
    max_sharpe=pc['Sharpe Ratio'].max()                                             #Best optimised portfolio wrt sharpe ratio
    max_sharpe_portfolio=pc.loc[pc['Sharpe Ratio'] == max_sharpe]
    #max_sharpe_portfolio.to_csv("Sharpe_MVO_dy_date.csv")
    opt_W=max_sharpe_portfolio.iloc[:,3:]
    opt_W=np.array(opt_W)

    #using opt weigths dynamically
    opt_W=np.array(opt_W)

    mvo_sharpe_list=[]
    df = pd.read_csv('n50.csv', parse_dates=['Date'], index_col='Date')
    df=df.loc[lb_start_date:lb_end_date]
    tdf = df.copy()  # deep copy
    df.reset_index(drop=True, inplace=True)
    trading_days = len(df) / number_of_years(tdf)
    #trading_days=1
    returnsh = df.pct_change()  # Here, returnsh would mean return considered for sharpe ratio
    returnsh.fillna(0, inplace=True)  # calculating daily returns of the stocks in the portfolio 
    covmatsh = returnsh.cov() * trading_days  # Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
    num_assets = len(df.columns)  # initializing number of stocks/assets considered in the portfolio
    risk_free_rate = 0.0358  #initializing risk free rate that will be used in calculating both the ratios (absolute value)
    # referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
    # In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
    # (improved)
    global_war_sharpe=[]
    sharpe_portfolio_return=[]
    sharpe_portfolio_risk=[]
    sharpe_portfolio_shratio=[]
    sharpe_portfolio_stockWeights=[]
    fitness=0
    weights=opt_W
    weights/=np.sum(opt_W)
    returns_temp = np.sum(returnsh.mean()*weights)*trading_days 
    varsh=np.dot(weights.T,np.dot(covmatsh,weights))   
    volatility_temp = np.sqrt(varsh)      
    fitness = ratio(returns_temp,volatility_temp,risk_free_rate)
    sharpe_portfolio_return.append(returns_temp)
    sharpe_portfolio_risk.append(volatility_temp)
    sharpe_portfolio_shratio.append(fitness)
    sharpe_portfolio_stockWeights.append(weights)
    sharpe_portfolio = {'Returns' : sharpe_portfolio_return, 'Standard Deviation' : sharpe_portfolio_risk,  'Sharpe Ratio' : sharpe_portfolio_shratio}  
    for counter,symbol in enumerate(df.columns):
      sharpe_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sharpe_portfolio_stockWeights]
    sharpe_pc = pd.DataFrame(sharpe_portfolio)
    sharpe_optimal=sharpe_pc.iloc[sharpe_pc['Sharpe Ratio'].idxmax()]
    sharpe_optimal=sharpe_optimal.to_frame()
    sharpe_optimal=sharpe_optimal.transpose()
    sharpe_optimal.loc[:, :] *= 100
    sharpe_optimal.loc[:, 'Sharpe Ratio'] /= 100
    to_append=[deciding_date,sharpe_optimal['Returns'].values[0],sharpe_optimal['Standard Deviation'].values[0],sharpe_optimal['Sharpe Ratio'].values[0]]
    mvo_sharpe_series = pd.Series(to_append, index = mvo_sharpe.columns)
    mvo_sharpe = mvo_sharpe.append(mvo_sharpe_series, ignore_index=True)
    sharpe_optimal.to_csv(str(deciding_date)+"sharpe_optimal.csv")
    return mvo_sharpe


In [ ]:
lb_start_date_index=df.index[df['Date'] == "2020-10-01"] 

In [ ]:
lb_end_date_index=df.index[df['Date']== "2020-12-31"]

In [ ]:
T_start_date_index=df.index[df['Date']=="2016-01-01"]

In [ ]:
T_end_date_index=df.index[df['Date']=="2020-09-30"]

In [ ]:
lb_start_date="2020-10-01"
lb_end_date="2020-12-31"
T_start_date="2016-01-01"
T_end_date="2020-09-30"

In [ ]:
while df.iloc[end_date_index].Date.values[0]:
    mvo_sharpe=MVO_Sharpe(df.iloc[T_start_date_index].Date.values[0],df.iloc[T_end_date_index].Date.values[0],
                          df.iloc[end_date_index+1].Date.values[0],mvo_sharpe,
                          df.iloc[lb_start_date_index].Date.values[0],df.iloc[lb_end_date_index].Date.values[0])
    lb_start_date_index+=1
    lb_end_date_index+=1
    T_start_date_index+=1
    T_end_date_index+=1

In [18]:
opt_W=np.array(opt_W)
opt_W

array([[0.90683447, 2.73032598, 2.41099729, 1.94431416, 1.09265762,
        8.2095824 , 1.20431415, 4.43309277, 1.03764767, 4.72604414,
        6.36985198, 1.89841337, 4.6416579 , 7.16667295, 4.12908314,
        7.00292116, 8.8294108 , 5.80183083, 3.40098081, 5.00633234,
        8.99704253, 2.58928725, 1.74780949, 3.72289478]])